In [40]:
import numpy as np 
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from scipy.linalg import sqrtm

# Inception Score   
<a href = 'https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiR6L3ehZ31AhWaxzgGHUhTB_UQFnoECBEQAw&url=https%3A%2F%2Farxiv.org%2Fpdf%2F1801.01973%23%3A~%3Atext%3DThe%2520Inception%2520Score%2520is%2520a%2Cfrom%2520the%2520CIFAR%252D10%2520dataset.&usg=AOvVaw16-0DfT4fxbaU2tFlxvcx5'>         A Note on the Inception Score</a>

In [2]:
def Kullbacks_leibler(py_x, eps = 1E-6):
    py = np.expand_dims(py_x.mean(0), 0)
    KL = py_x * np.log((py_x + eps) / (py + eps))
    return np.sum(KL, axis = 1)

In [3]:
# Test code
py_x = np.array([[0.3, 0.3, 0.4], [0.1, 0.2, 0.7], [0.0, 0.0, 1.0]])
Kullbacks_leibler(py_x)

array([0.19576713, 0.00769615, 0.35667452])

In [4]:
def Inception_score(image, n_split = 10):
    scores = [] 
    model = InceptionV3()
    n_parts = int(np.floor(image.shape[0] / n_split ))
    image = tf.image.resize(image, (299,299)) /255.
    for i in range(n_split):
        ix_start, ix_end = i * n_parts, i * n_parts + n_parts
        img = image[ix_start:ix_end]
        py_x = model.predict(img)
        scores.append(np.exp(Kullbacks_leibler(py_x).mean()))
    return np.mean(scores), np.std(scores)

In [6]:
# Test code
(x,_),(_,_) = tf.keras.datasets.cifar10.load_data()
Inception_score(x[:1000])

(7.4813323, 0.36871633)

# FID
<a href = 'https://arxiv.org/pdf/1706.08500.pdf'>         GANs Trained by a Two Time-Scale Update Rule
Converge to a Local Nash Equilibrium</a>
<br>
d^2 = ||mu_1 – mu_2||^2 + Tr(C_1 + C_2 – 2*sqrt(C_1*C_2))

In [41]:
def fid_score(act1, act2):
    mu1, sigma1 = act1.mean(axis = 0), np.cov(act1, rowvar= False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean =sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

In [42]:
# Test code
act1 = np.random.random((10,2048))
act2 = np.random.random((10,2048))
fid_score(act1,act2)

355.8212503991989

In [44]:
def calculate_fid_score(image1, image2, inp_shape = (150,150)):
    model = InceptionV3(include_top= False, input_shape= (*inp_shape,3),pooling= 'avg')
    image1 = tf.image.resize(image1, inp_shape) /255.
    image2 = tf.image.resize(image2, inp_shape) /255.
    act1 = model.predict(image1)
    act2 = model.predict(image2)
    return fid_score(act1,act2)

In [45]:
# Test code
image1 = np.random.randint(0,255, size = (10,300,300,3))
image2 = np.random.randint(0,255, size = (10,300,300,3))
calculate_fid_score(image1, image2)

20.966887940713626